In [6]:
import os
import sys
import math
import joblib
import FE_0823 as FE
import numpy as np
import pandas as pd
from tqdm import tqdm

In [107]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [3]:
sale = pd.read_excel(os.path.join('..', '..', '0.Data', '01_제공데이터', 'sale_data_v05_0828.xlsx'))
meta = pd.read_excel(os.path.join('..', '..', '0.Data', '01_제공데이터', 'sale_meta_v04_0823.xlsx'))
item = pd.read_excel(os.path.join('..', '..', '0.Data', '01_제공데이터', 'item_meta_v04_0828.xlsx'))

In [7]:
sale['방송월'] = sale['방송일시'].dt.month
sale['방송시간(시간)'] = sale['방송일시'].dt.hour
sale['방송시간(분)'] = sale['방송일시'].dt.minute
sale['판매량'] = sale['취급액'] / sale['판매단가']
sale['판매량'] = sale['판매량'].fillna(0).apply(lambda x : math.ceil(x))

In [42]:
temp = pd.pivot_table(sale, index = '상품군', columns = '방송월', values = '판매량', aggfunc = np.mean).T.reset_index()
temp.columns = [temp.columns[0]] + list(map(lambda x :  x, temp.columns[1:]))

In [49]:
temp

,방송월,가구,가전,건강기능,농수축,무형,생활용품,속옷,의류,이미용,잡화,주방,침구
0,1,nan,16.060,223.095,939.365,0.000,255.373,348.560,311.944,885.581,179.888,158.374,nan
1,2,22.850,16.447,377.359,931.847,0.000,242.468,221.562,365.883,782.789,414.522,174.450,nan
2,3,17.336,13.691,361.767,873.856,0.000,291.557,271.208,314.484,859.473,452.903,159.489,nan
3,4,16.043,10.907,320.351,941.578,0.000,166.069,367.310,363.860,714.286,291.408,180.629,62.647
4,5,14.774,11.348,274.167,925.267,0.000,218.547,452.833,432.837,839.358,206.592,194.441,156.716
5,6,15.140,13.523,458.644,981.081,0.000,304.833,431.297,417.326,813.386,248.217,198.430,226.099
6,7,18.758,14.020,409.159,928.225,0.000,152.562,334.398,464.096,782.589,213.727,206.989,170.404
7,8,19.082,17.194,266.479,965.266,0.000,106.270,538.045,575.944,491.321,155.831,185.643,160.750
8,9,22.015,19.009,384.753,987.361,0.000,139.324,273.143,292.472,637.667,181.878,178.756,44.000
9,10,22.324,22.274,389.010,985.230,0.000,190.256,414.382,326.192,704.338,204.249,180.002,114.412


In [47]:
sale['상품군별월별평균판매량'] = None
for i in range(1, 13):
    for cate in sale['상품군'].unique():
        sale.loc[(sale['방송월'] == i) & (sale['상품군'] == cate), '상품군별월별평균판매량'] = temp[cate].loc[temp['방송월'] == i].values[0]

In [56]:
temp = pd.pivot_table(sale, index = '상품군', columns = '방송시간(시간)', values = '판매량', aggfunc = np.mean).T.reset_index()
temp.columns = [temp.columns[0]] + list(map(lambda x : x, temp.columns[1:]))
temp

,방송시간(시간),가구,가전,건강기능,농수축,무형,생활용품,속옷,의류,이미용,잡화,주방,침구
0,0,16.746,10.696,278.476,1991.833,0.000,111.761,302.576,262.046,686.841,115.301,94.761,45.250
1,1,65.684,4.643,112.405,469.500,0.000,113.105,172.018,199.721,264.881,114.442,77.027,28.853
2,2,2.500,nan,171.200,693.750,0.000,150.333,299.800,246.556,391.000,123.875,204.333,74.000
3,6,4.417,nan,208.010,304.700,0.000,176.776,277.036,316.961,276.080,254.942,189.954,75.889
4,7,4.550,7.357,370.763,611.222,0.000,216.545,410.076,481.649,527.658,392.188,361.511,nan
5,8,14.900,58.667,457.345,708.200,0.000,210.227,430.350,530.511,720.000,207.625,242.000,51.667
6,9,nan,11.071,488.674,1032.692,0.000,271.820,548.299,509.196,779.115,256.261,331.579,nan
7,10,11.667,11.959,630.682,982.516,0.000,239.500,529.275,420.322,965.974,207.800,182.717,241.111
8,11,nan,17.172,426.364,960.415,0.000,148.353,589.523,358.628,875.167,198.624,184.069,200.104
9,12,55.111,22.758,360.968,824.868,0.000,344.071,496.912,370.784,819.440,259.154,167.736,172.837


In [62]:
sale['상품군별시간대별평균판매량'] = None
for i in range(24):
    for cate in sale['상품군'].unique():
        try:
            sale.loc[(sale['방송시간(시간)'] == i) & (sale['상품군'] == cate), '상품군별시간대별평균판매량'] = temp[cate].loc[temp['방송시간(시간)'] == i].values[0]
        except:
            continue

In [67]:
temp = pd.pivot_table(sale, index = '상품군', columns = '방송시간(분)', values = '판매량', aggfunc = np.mean).T.reset_index()
temp.columns = [temp.columns[0]] + list(map(lambda x : x, temp.columns[1:]))
temp

,방송시간(분),가구,가전,건강기능,농수축,무형,생활용품,속옷,의류,이미용,잡화,주방,침구
0,0,13.048,12.200,303.153,792.737,0.000,136.948,273.263,288.507,709.007,155.646,139.846,99.325
1,5,nan,29.042,nan,1050.000,nan,nan,nan,nan,nan,1026.000,nan,nan
2,10,nan,nan,nan,880.905,nan,256.200,421.452,57.283,194.500,356.118,nan,nan
3,15,nan,17.292,nan,612.167,nan,nan,nan,nan,nan,nan,nan,nan
4,20,15.534,13.764,354.275,963.700,0.000,161.428,328.253,390.907,746.075,278.892,172.568,158.019
5,30,5.318,10.651,172.333,791.516,0.000,165.287,688.223,248.141,896.762,49.419,380.005,115.267
6,35,nan,15.083,nan,796.000,nan,nan,nan,nan,nan,839.000,nan,nan
7,40,25.487,21.578,469.616,1167.815,0.000,229.824,425.525,518.489,959.210,407.464,244.063,212.498
8,45,nan,28.833,nan,799.000,nan,nan,nan,nan,nan,nan,nan,nan
9,50,6.538,20.803,198.500,1060.636,0.000,201.450,707.472,157.039,254.250,92.318,341.583,nan


In [69]:
sale['상품군별시간분별평균판매량'] = None
for i in sale['방송시간(분)'].unique():
    for cate in sale['상품군'].unique():
        try:
            sale.loc[(sale['방송시간(분)'] == i) & (sale['상품군'] == cate), '상품군별시간분별평균판매량'] = temp[cate].loc[temp['방송시간(분)'] == i].values[0]
        except:
            continue

In [74]:
sale['상품군별시간분별평균판매량'].sort_values()

38308      0.000
13805      0.000
13806      0.000
13807      0.000
25796      0.000
          ...   
1529    1167.815
30783   1167.815
26725   1167.815
4318    1167.815
24853   1167.815
Name: 상품군별시간분별평균판매량, Length: 38309, dtype: object

In [120]:
df_eco = pd.read_excel(os.path.join('..', '..', '0.Data', '03_외부데이터', '전처리', 'prep_2차외부데이터_0908.xlsx'))

In [121]:
from sklearn.decomposition import PCA

In [145]:
pca = PCA(n_components = 0.9)

In [151]:
X = pca.fit_transform(df_eco.iloc[:,35:])

In [152]:
pd.concat([df_eco.iloc[:,:35], pd.DataFrame(X, columns = ['pca_1',  'pca_2'])], axis = 1)

,날짜,경상지수,불변지수,년도,월,현재생활형편CSI,현재경기판단CSI,생활형편전망CSI,소비지출전망CSI,주택가격전망CSI,...,자동차 및 자동차용품,가구,애완용품,여행 및 교통서비스,문화 및 레저서비스,e쿠폰서비스,음식서비스,기타서비스,pca_1,pca_2
0,2019-01-01,142.200,139.300,2019,1,90,65,91,109,91,...,95625,279933,71584,1506113,163249,236089,623920,80302,-9212.253,-46576.914
1,2019-02-01,116.100,113.300,2019,2,93,70,92,109,84,...,78913,269714,63859,1381332,174886,230314,616842,59239,403203.731,-79624.484
2,2019-03-01,137.000,134.000,2019,3,91,70,94,110,83,...,105998,305141,71619,1345140,181135,253530,712110,63577,22589.059,35087.332
3,2019-04-01,133.500,130.100,2019,4,93,74,95,110,87,...,96346,283592,74272,1359831,179770,222461,687613,62386,77281.527,-67004.352
4,2019-05-01,137.100,133.800,2019,5,91,69,92,109,93,...,105237,284614,75834,1452693,174178,271802,743966,75738,-98100.481,-67224.697
5,2019-06-01,125.800,123.400,2019,6,91,69,92,108,97,...,97677,268513,76382,1441717,225380,256486,775346,73099,72959.078,-111293.654
6,2019-07-01,127.200,124.700,2019,7,91,67,92,107,106,...,96062,261562,76562,1606012,195490,278475,828720,84203,-105215.575,-75430.926
7,2019-08-01,123.100,120.300,2019,8,90,63,89,105,107,...,110450,274217,75856,1625159,197185,304146,914251,113270,-38916.193,-76388.513
8,2019-09-01,127.600,124.300,2019,9,92,68,92,106,109,...,168561,286636,80834,1315828,153186,272024,850015,159365,54311.143,-70163.835
9,2019-10-01,143.000,139.000,2019,10,92,72,93,108,115,...,105358,319898,86880,1433122,187668,296613,907731,105421,-130323.258,-41869.683
